In [1]:
import numpy as np
import pandas as pd
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
from platform import python_version
from metrics_as_scores.distribution.distribution import KDE_approx
from metrics_as_scores.tools.funcs import transform_to_MAS_dataset
import scipy.stats as stats
from sklearn.preprocessing import PowerTransformer
import itertools

In [2]:
pd.set_option('display.max_rows', None)

# Sonarqube

In [3]:
sq = pd.read_csv('Sonarqube_results.csv')

In [4]:
pd.set_option('display.max_columns', None)
sq

,Project,bugs,reliability_rating,reliability_remediation_effort,vulnerabilities,security_rating,security_remediation_effort,security_hotspots,security_review_rating,code_smells,sqale_rating,sqale_index,sqale_debt_ratio,files,functions,statements,lines,comment_lines,duplicated_files,duplicated_lines,duplicated_lines_density,complexity,cognitive_complexity,violations
0,ant-1.8,404,5,84.966667,17,5,5.750000,86,5,9092,1,1431.033333,1.6,1725,13160,59383,321575,57876,156,10111,3.1,26923,22931,9513
1,antlr-3,106,5,14.950000,0,1,0.000000,80,5,6669,1,934.400000,2.0,1421,7613,44089,200032,22153,94,11499,5.7,14587,9305,6775
2,aoi-2.8,1953,5,450.116667,5,5,1.416667,165,5,12125,1,1606.300000,2.6,575,6839,67724,165461,14285,226,27481,16.6,23623,28934,14083
3,argouml,493,5,59.650000,8,5,1.916667,60,5,17160,1,2277.266667,1.0,3001,18026,89295,752737,106516,352,25667,3.4,42765,37119,17661
4,aspectj,859,5,144.133333,8,5,2.000000,193,5,44644,1,6653.583333,2.7,2929,43595,271309,786915,146652,1514,310623,39.5,129113,151894,45511
5,axion-1,125,5,15.816667,0,1,0.000000,8,5,2978,1,320.850000,2.6,262,3000,11593,43651,3394,34,1595,3.7,6224,5343,3103
6,azureus,1317,5,226.250000,42,5,14.783333,482,5,53477,1,5155.616667,2.1,3268,35670,212707,794681,47432,490,32075,4.0,86381,104075,54836
7,batik-1,543,5,99.583333,0,1,0.000000,105,5,14055,1,2999.200000,2.8,1851,16568,90615,399369,62443,323,25802,6.5,38324,34866,14598
8,castor-,88540,5,22706.366670,6,5,2.000000,234,5,250068,2,21213.083330,5.6,3414,6120,39288,944219,78414,2139,371888,39.4,15711,14227,338614
9,cayenne,1080,5,148.716667,0,1,0.000000,147,5,22622,1,2877.066667,1.4,4868,23500,117840,696186,84388,955,88488,12.7,46324,36577,23702


# Codiga

In [5]:
co = pd.read_csv('Codiga_proc_results.csv')

In [6]:
co = co.rename(columns={"Quality Score": "Quality Score(%)","Long function": "Long function(%)", "Complex function": "Complex function(%)", "Duplicated lines":"Duplicated lines(%)"})

In [7]:
co['Quality Score(%)'] = co['Quality Score(%)'].str.rstrip('%').astype('float')

In [8]:
co['Long function(%)'] = co['Long function(%)'].str.rstrip('%').astype('float')

In [9]:
co['Complex function(%)'] = co['Complex function(%)'].str.rstrip('%').astype('float')

In [10]:
co['Duplicated lines(%)'] = co['Duplicated lines(%)'].str.rstrip('%').astype('float')

In [11]:
co

,Project,Quality Score(%),Grade,LOC,Errors reported,Violations,Informational,Error,Warning,Critical,Long function(%),Complex function(%),Duplicated lines(%),Outdated,To upgrade,Up to Date,Total Cost,Time
0,ant-1.8,94.0,A,254244,4213,3691,189,189,3218,95,4.0,1.0,0.0,0,0,0,47429,677.56
1,antlr-3,93.0,A,209464,3582,3184,499,968,1405,312,5.0,1.0,2.0,0,0,0,51590,737.00
2,aoi-2.8,85.0,A,161464,2972,2083,275,159,1514,135,10.0,2.0,9.0,0,0,0,49186,702.66
3,argouml,95.0,A,403693,5270,4440,156,281,3736,267,5.0,1.0,0.0,0,0,0,69465,992.36
4,aspectj,88.0,A,783960,19687,17330,1648,770,14205,707,8.0,2.0,0.0,0,0,0,220115,3144.50
5,axion-1,85.0,A,42522,1346,1256,26,29,1136,65,3.0,1.0,2.0,0,0,0,13815,197.36
6,azureus,93.0,A,790867,14526,11430,681,752,9546,451,10.0,1.0,0.0,0,0,0,204127,2916.10
7,batik-1,90.0,A,370579,8981,8052,302,437,6983,330,6.0,1.0,0.0,0,0,0,100559,1436.56
8,castor-,95.0,A,505617,6633,55811,312,317,4814,138,5.0,1.0,0.0,0,0,0,80896,1155.66
9,cayenne,97.0,A,391517,4021,3288,157,205,2625,301,4.0,1.0,4.0,0,0,0,58118,830.26


# Code Climate

In [12]:
cc = pd.read_csv('Code_climate_results.csv')

In [13]:
cc

,Project,Files,LOC,Code smell,Duplication,Issues,Maintainibility score,TD time (hrs),TD ratio (%)
0,ant-1.8,3044,103163,2169,758,2927,C,5595.5,18.7
1,antlr-3,1600,82523,1194,986,2180,D,5701.4,24.1
2,aoi-2.8,1915,111440,2702,2470,5172,D,10842.6,33.4
3,argouml,6241,197553,4047,2428,6475,D,11998.6,20.2
4,aspectj,10951,500395,16165,6697,22862,D,67848.8,43.1
5,axion-1,540,24079,663,322,985,D,1562.4,24.0
6,azureus,11329,494116,8948,3911,12859,C,30520.2,19.7
7,batik-1,5647,190423,3751,2720,6471,D,12370.0,21.7
8,castor-,3709,81099,1355,935,2290,D,5376.4,23.1
9,cayenne,5426,151214,2919,2071,4990,D,10111.4,22.6


# QCC Software metrics

In [14]:
sm = pd.read_csv('org-data.csv')

In [15]:
df = sm[sm.System != 'netbean']
df = df[df.System != 'eclipse']

In [16]:
# df

In [17]:
test = df.loc[df['Metric'] == 'MLOC']

In [18]:
test.head(20)

,System,Metric,Value,Domain,System_org
41046,ant-1.8,MLOC,166.0,parsers/generators/make,ant
41047,ant-1.8,MLOC,159.0,parsers/generators/make,ant
41048,ant-1.8,MLOC,149.0,parsers/generators/make,ant
41049,ant-1.8,MLOC,148.0,parsers/generators/make,ant
41050,ant-1.8,MLOC,146.0,parsers/generators/make,ant
41051,ant-1.8,MLOC,145.0,parsers/generators/make,ant
41052,ant-1.8,MLOC,145.0,parsers/generators/make,ant
41053,ant-1.8,MLOC,143.0,parsers/generators/make,ant
41054,ant-1.8,MLOC,137.0,parsers/generators/make,ant
41055,ant-1.8,MLOC,133.0,parsers/generators/make,ant


## Unique SM

In [19]:
um = df['Metric'].unique()
print(um)

['LCOM' 'DIT' 'CE' 'NOP' 'RMI' 'NOM' 'NSF' 'NSM' 'TLOC' 'NBD' 'SIX' 'CA'
 'NOI' 'PAR' 'WMC' 'NOC' 'RMA' 'NORM' 'MLOC' 'NSC' 'RMD' 'NOF' 'VG']


## Unique Projects

In [20]:
sys = df['System'].unique()
print(sys)

['ant-1.8' 'antlr-3' 'aoi-2.8' 'argouml' 'aspectj' 'axion-1' 'azureus'
 'batik-1' 'castor-' 'cayenne' 'checkst' 'cobertu' 'collect' 'colt-1.'
 'columba' 'compier' 'c_jdbc-' 'derby-1' 'display' 'drawswf' 'drjava-'
 'emma-2.' 'exoport' 'findbug' 'fitjava' 'fitlibr' 'freecol' 'freecs-'
 'freemin' 'galleon' 'ganttpr' 'geotool' 'hadoop-' 'heritri' 'hiberna'
 'hsqldb-' 'htmluni' 'informa' 'iReport' 'itext-5' 'ivatagr' 'jag-6.1'
 'james-2' 'jasml-0' 'jasperr' 'javacc-' 'jboss-5' 'jchempa' 'jedit-4'
 'jena-2.' 'jext-5.' 'jFin_Da' 'jfreech' 'jgraph-' 'jgraphp' 'jgrapht'
 'jgroups' 'jhotdra' 'jmeter-' 'jmoney-' 'joggpla' 'jparse-' 'jpf-1.5'
 'jrat-1-' 'jre-1.6' 'jrefact' 'jruby-1' 'jspwiki' 'jsXe-04' 'jtopen-'
 'jung-2.' 'junit-4' 'log4j-2' 'lucene-' 'marauro' 'maven-3' 'megamek'
 'mvnforu' 'myfaces' 'nakedob' 'nekohtm' 'openjms' 'oscache' 'picocon'
 'pmd-4.2' 'poi-3.6' 'pooka-3' 'proguar' 'quartz-' 'quickse' 'quilt-0'
 'roller-' 'rssowl-' 'sablecc' 'sandmar' 'springf' 'squirre' 'struts-'
 'sunf

## Unique Domains

In [21]:
df['Domain'].unique()

array(['parsers/generators/make', '3D/graphics/media',
       'diagram generator/data visualization', 'programming language',
       'database', 'middleware', 'IDE', 'testing', 'tool', 'SDK', 'games'],
      dtype=object)

In [22]:
# 'LCOM', 'DIT', 'CE', 'NOP', 'RMI', 'NOM', 'NSF', 'NSM', 'TLOC',
#        'NBD', 'SIX', 'CA', 'NOI', 'PAR', 'WMC', 'NOC', 'RMA', 'NORM',
#        'MLOC', 'NSC', 'RMD', 'NOF', 'VG'

## Final Qcc data

In [23]:
Qcc = []
for i in(sys):
    test = []
    test = df.loc[df['System']== i].groupby('Metric')['Value'].sum()

    Qcc.append({"Project": i,'Domain' : df['Domain'].loc[df['System']== i].unique()[0],'NOP':test['NOP'],'TLOC':test['TLOC']})
#                 ,
#                'LCOM':test['LCOM'],'DIT':test['DIT'],'CE':test['CE'],'RMI':test['RMI'],'NOM':test['NOM'],'NSF':test['NSF'],
#                'NSM':test['NSM'],'NBD':test['NBD'],'SIX':test['SIX'],'CA':test['CA'],'NOI':test['NOI'],'PAR':test['PAR'],
#                'WMC':test['WMC'],'NOC':test['NOC'],'RMA':test['RMA'],'NORM':test['NORM'],'MLOC':test['MLOC'],'NSC':test['NSC'],
#                'RMD':test['RMD'],'NOF':test['NOF'],'VG':test['VG']})
Qcc_df = pd.DataFrame.from_records(Qcc)
Qcc_df.to_csv('Qcc_proc_data.csv',index=False)

In [24]:
Qcc_df

,Project,Domain,NOP,TLOC
0,ant-1.8,parsers/generators/make,127.0,127559.0
1,antlr-3,parsers/generators/make,20.0,47443.0
2,aoi-2.8,3D/graphics/media,23.0,110009.0
3,argouml,diagram generator/data visualization,77.0,105806.0
4,aspectj,programming language,144.0,501762.0
5,axion-1,database,13.0,24163.0
6,azureus,database,473.0,495532.0
7,batik-1,3D/graphics/media,117.0,194715.0
8,castor-,middleware,381.0,219681.0
9,cayenne,database,235.0,198176.0


'LCOM', 'DIT', 'CE', 'RMI', 'NOM', 'NSF', 'NSM',
'NBD', 'SIX', 'CA', 'NOI', 'PAR', 'WMC', 'NOC', 'RMA', 'NORM',
'MLOC', 'NSC', 'RMD', 'NOF', 'VG'

In [25]:
scores = []
# val = 'ant-1.8'
for val in sys:
    sys_df = df[df.System == val]
# scores.append({"Project": val})
    LCOM_df = sys_df[sys_df.Metric == 'LCOM']
    #         # do not transform
    LCOM_01 = LCOM_df.Value.quantile(.01)
    LCOM_20 = LCOM_df.Value.quantile(.20)
    LCOM_40 = LCOM_df.Value.quantile(.40)
    LCOM_50 = LCOM_df.Value.quantile(.50)
    LCOM_60 = LCOM_df.Value.quantile(.60)
    LCOM_80 = LCOM_df.Value.quantile(.80)
    LCOM_99 = LCOM_df.Value.quantile(.99)
    LCOM_std = LCOM_df.Value.std()

    DIT_df = sys_df[sys_df.Metric == 'DIT']
    DIT_01 = DIT_df.Value.quantile(.01)
    DIT_20 = DIT_df.Value.quantile(.20)
    DIT_40 = DIT_df.Value.quantile(.40)
    DIT_50 = DIT_df.Value.quantile(.50)
    DIT_60 = DIT_df.Value.quantile(.60)
    DIT_80 = DIT_df.Value.quantile(.80)
    DIT_99 = DIT_df.Value.quantile(.99)
    DIT_std = DIT_df.Value.std()

    CE_df = sys_df[sys_df.Metric == 'CE']
    CE_01 = CE_df.Value.quantile(.01)
    CE_20 = CE_df.Value.quantile(.20)
    CE_40 = CE_df.Value.quantile(.40)
    CE_50 = CE_df.Value.quantile(.50)
    CE_60 = CE_df.Value.quantile(.60)
    CE_80 = CE_df.Value.quantile(.80)
    CE_99 = CE_df.Value.quantile(.99)
    CE_std = CE_df.Value.std()

    RMI_df = sys_df[sys_df.Metric == 'RMI']
    RMI_01 = RMI_df.Value.quantile(.01)
    RMI_20 = RMI_df.Value.quantile(.20)
    RMI_40 = RMI_df.Value.quantile(.40)
    RMI_50 = RMI_df.Value.quantile(.50)
    RMI_60 = RMI_df.Value.quantile(.60)
    RMI_80 = RMI_df.Value.quantile(.80)
    RMI_99 = RMI_df.Value.quantile(.99)
    RMI_std = RMI_df.Value.std()

    NOM_df = sys_df[sys_df.Metric == 'NOM']
    NOM_01 = NOM_df.Value.quantile(.01)
    NOM_20 = NOM_df.Value.quantile(.20)
    NOM_40 = NOM_df.Value.quantile(.40)
    NOM_50 = NOM_df.Value.quantile(.50)
    NOM_60 = NOM_df.Value.quantile(.60)
    NOM_80 = NOM_df.Value.quantile(.80)
    NOM_99 = NOM_df.Value.quantile(.99)
    NOM_std = LCOM_df.Value.std() 

    NSF_df = sys_df[sys_df.Metric == 'NSF']
    NSF_01 = NSF_df.Value.quantile(.01)
    NSF_20 = NSF_df.Value.quantile(.20)
    NSF_40 = NSF_df.Value.quantile(.40)
    NSF_50 = NSF_df.Value.quantile(.50)
    NSF_60 = NSF_df.Value.quantile(.60)
    NSF_80 = NSF_df.Value.quantile(.80)
    NSF_99 = NSF_df.Value.quantile(.99)
    NSF_std = NSF_df.Value.std()

    NSM_df = sys_df[sys_df.Metric == 'NSM']
    NSM_01 = NSM_df.Value.quantile(.01)
    NSM_20 = NSM_df.Value.quantile(.20)
    NSM_40 = NSM_df.Value.quantile(.40)
    NSM_50 = NSM_df.Value.quantile(.50)
    NSM_60 = NSM_df.Value.quantile(.60)
    NSM_80 = NSM_df.Value.quantile(.80)
    NSM_99 = NSM_df.Value.quantile(.99)
    NSM_std = NSM_df.Value.std()

    NBD_df = sys_df[sys_df.Metric == 'NBD']
    NBD_01 = NBD_df.Value.quantile(.01)
    NBD_20 = NBD_df.Value.quantile(.20)
    NBD_40 = NBD_df.Value.quantile(.40)
    NBD_50 = NBD_df.Value.quantile(.50)
    NBD_60 = NBD_df.Value.quantile(.60)
    NBD_80 = NBD_df.Value.quantile(.80)
    NBD_99 = NBD_df.Value.quantile(.99)
    NBD_std = NBD_df.Value.std()

    SIX_df = sys_df[sys_df.Metric == 'SIX']
    SIX_01 = SIX_df.Value.quantile(.01)
    SIX_20 = SIX_df.Value.quantile(.20)
    SIX_40 = SIX_df.Value.quantile(.40)
    SIX_50 = SIX_df.Value.quantile(.50)
    SIX_60 = SIX_df.Value.quantile(.60)
    SIX_80 = SIX_df.Value.quantile(.80)
    SIX_99 = SIX_df.Value.quantile(.99)
    SIX_std = SIX_df.Value.std()

    CA_df = sys_df[sys_df.Metric == 'CA']
    CA_01 = CA_df.Value.quantile(.01)
    CA_20 = CA_df.Value.quantile(.20)
    CA_40 = CA_df.Value.quantile(.40)
    CA_50 = CA_df.Value.quantile(.50)
    CA_60 = CA_df.Value.quantile(.60)
    CA_80 = CA_df.Value.quantile(.80)
    CA_99 = CA_df.Value.quantile(.99)
    CA_std = CA_df.Value.std()

    NOI_df = sys_df[sys_df.Metric == 'NOI']
    NOI_01 = NOI_df.Value.quantile(.01)
    NOI_20 = NOI_df.Value.quantile(.20)
    NOI_40 = NOI_df.Value.quantile(.40)
    NOI_50 = NOI_df.Value.quantile(.50)
    NOI_60 = NOI_df.Value.quantile(.60)
    NOI_80 = NOI_df.Value.quantile(.80)
    NOI_99 = NOI_df.Value.quantile(.99)
    NOI_std = NOI_df.Value.std()

    PAR_df = sys_df[sys_df.Metric == 'PAR']
    PAR_01 = PAR_df.Value.quantile(.01)
    PAR_20 = PAR_df.Value.quantile(.20)
    PAR_40 = PAR_df.Value.quantile(.40)
    PAR_50 = PAR_df.Value.quantile(.50)
    PAR_60 = PAR_df.Value.quantile(.60)
    PAR_80 = PAR_df.Value.quantile(.80)
    PAR_99 = PAR_df.Value.quantile(.99)
    PAR_std = PAR_df.Value.std()

    WMC_df = sys_df[sys_df.Metric == 'WMC']
    WMC_01 = WMC_df.Value.quantile(.01)
    WMC_20 = WMC_df.Value.quantile(.20)
    WMC_40 = WMC_df.Value.quantile(.40)
    WMC_50 = WMC_df.Value.quantile(.50)
    WMC_60 = WMC_df.Value.quantile(.60)
    WMC_80 = WMC_df.Value.quantile(.80)
    WMC_99 = WMC_df.Value.quantile(.99)
    WMC_std = WMC_df.Value.std()

    NOC_df = sys_df[sys_df.Metric == 'NOC']
    NOC_01 = NOC_df.Value.quantile(.01)
    NOC_20 = NOC_df.Value.quantile(.20)
    NOC_40 = NOC_df.Value.quantile(.40)
    NOC_50 = NOC_df.Value.quantile(.50)
    NOC_60 = NOC_df.Value.quantile(.60)
    NOC_80 = NOC_df.Value.quantile(.80)
    NOC_99 = NOC_df.Value.quantile(.99)
    NOC_std = NOC_df.Value.std()

    RMA_df = sys_df[sys_df.Metric == 'RMA']
    RMA_01 = RMA_df.Value.quantile(.01)
    RMA_20 = RMA_df.Value.quantile(.20)
    RMA_40 = RMA_df.Value.quantile(.40)
    RMA_50 = RMA_df.Value.quantile(.50)
    RMA_60 = RMA_df.Value.quantile(.60)
    RMA_80 = RMA_df.Value.quantile(.80)
    RMA_99 = RMA_df.Value.quantile(.99)
    RMA_std = RMA_df.Value.std()

    NORM_df = sys_df[sys_df.Metric == 'NORM']
    NORM_01 = NORM_df.Value.quantile(.01)
    NORM_20 = NORM_df.Value.quantile(.20)
    NORM_40 = NORM_df.Value.quantile(.40)
    NORM_50 = NORM_df.Value.quantile(.50)
    NORM_60 = NORM_df.Value.quantile(.60)
    NORM_80 = NORM_df.Value.quantile(.80)
    NORM_99 = NORM_df.Value.quantile(.99)
    NORM_std = NORM_df.Value.std()

    MLOC_df = sys_df[sys_df.Metric == 'MLOC']
    MLOC_01 = MLOC_df.Value.quantile(.01)
    MLOC_20 = MLOC_df.Value.quantile(.20)
    MLOC_40 = MLOC_df.Value.quantile(.40)
    MLOC_50 = MLOC_df.Value.quantile(.50)
    MLOC_60 = MLOC_df.Value.quantile(.60)
    MLOC_80 = MLOC_df.Value.quantile(.80)
    MLOC_99 = MLOC_df.Value.quantile(.99)
    MLOC_std = MLOC_df.Value.std()

    NSC_df = sys_df[sys_df.Metric == 'NSC']
    NSC_01 = NSC_df.Value.quantile(.01)
    NSC_20 = NSC_df.Value.quantile(.20)
    NSC_40 = NSC_df.Value.quantile(.40)
    NSC_50 = NSC_df.Value.quantile(.50)
    NSC_60 = NSC_df.Value.quantile(.60)
    NSC_80 = NSC_df.Value.quantile(.80)
    NSC_99 = NSC_df.Value.quantile(.99)
    NSC_std = NSC_df.Value.std()

    RMD_df = sys_df[sys_df.Metric == 'RMD']
    RMD_01 = RMD_df.Value.quantile(.01)
    RMD_20 = RMD_df.Value.quantile(.20)
    RMD_40 = RMD_df.Value.quantile(.40)
    RMD_50 = RMD_df.Value.quantile(.50)
    RMD_60 = RMD_df.Value.quantile(.60)
    RMD_80 = RMD_df.Value.quantile(.80)
    RMD_99 = RMD_df.Value.quantile(.99)
    RMD_std = RMD_df.Value.std()

    NOF_df = sys_df[sys_df.Metric == 'NOF']
    NOF_01 = NOF_df.Value.quantile(.01)
    NOF_20 = NOF_df.Value.quantile(.20)
    NOF_40 = NOF_df.Value.quantile(.40)
    NOF_50 = NOF_df.Value.quantile(.50)
    NOF_60 = NOF_df.Value.quantile(.60)
    NOF_80 = NOF_df.Value.quantile(.80)
    NOF_99 = NOF_df.Value.quantile(.99)
    NOF_std = NOF_df.Value.std()

    VG_df = sys_df[sys_df.Metric == 'VG']
    VG_01 = VG_df.Value.quantile(.01)
    VG_20 = VG_df.Value.quantile(.20)
    VG_40 = VG_df.Value.quantile(.40)
    VG_50 = VG_df.Value.quantile(.50)
    VG_60 = VG_df.Value.quantile(.60)
    VG_80 = VG_df.Value.quantile(.80)
    VG_99 = VG_df.Value.quantile(.99)
    VG_std = VG_df.Value.std()

    scores.append({"Project":val,"LCOM_01": LCOM_01, "LCOM_20": LCOM_20,"LCOM_40": LCOM_40,"LCOM_50": LCOM_50,"LCOM_60": LCOM_60, 
                   "LCOM_80": LCOM_80,"LCOM_99": LCOM_99,"LCOM_std": LCOM_std,"DIT_01": DIT_01, "DIT_20": DIT_20,"DIT_40": DIT_40,
                   "DIT_50": DIT_50,"DIT_60": DIT_60,"DIT_80": DIT_80,"DIT_99": DIT_99,"DIT_std": DIT_std, "CE_01": CE_01, 
                   "CE_20": CE_20,"CE_40": CE_40,"CE_50": CE_50,"CE_60": CE_60,"CE_80": CE_80,"CE_99": CE_99,
                   "CE_std": CE_std, "RMI_01": RMI_01, "RMI_20": RMI_20,"RMI_40": RMI_40,"RMI_50": RMI_50,"RMI_60": RMI_60,
                   "RMI_80": RMI_80,"RMI_99": RMI_99,"RMI_std": RMI_std, "NOM_01": NOM_01, "NOM_20": NOM_20,"NOM_40": NOM_40,
                   "NOM_50": NOM_50,"NOM_60": NOM_60,"NOM_80": NOM_80,"NOM_99": NOM_99,"NOM_std": NOM_std, "NSF_01": NSF_01,
                   "NSF_20": NSF_20,"NSF_40": NSF_40,"NSF_50": NSF_50,"NSF_60": NSF_60,"NSF_80": NSF_80,"NSF_99": NSF_99,
                   "NSF_std": NSF_std, "NSM_01": NSM_01, "NSM_20": NSM_20,"NSM_40": NSM_40,"NSM_50": NSM_50,"NSM_60": NSM_60,
                   "NSM_80": NSM_80,"NSM_99": NSM_99,"NSM_std": NSM_std, "NBD_01": NBD_01, "NBD_20": NBD_20,"NBD_40": NBD_40,
                   "NBD_50": NBD_50,"NBD_60": NBD_60,"NBD_80": NBD_80,"NBD_99": NBD_99,"NBD_std": NBD_std, "SIX_01": SIX_01, 
                   "SIX_20": SIX_20,"SIX_40": SIX_40,"SIX_50": SIX_50,"SIX_60": SIX_60,"SIX_80": SIX_80,"SIX_99": SIX_99,
                   "SIX_std": SIX_std, "CA_01": CA_01, "CA_20": CA_20,"CA_40": CA_40,"CA_50": CA_50,"CA_60": CA_60,
                   "CA_80": CA_80,"CA_99": CA_99,"CA_std": CA_std, "NOI_01": NOI_01, "NOI_20": NOI_20,"NOI_40": NOI_40,
                   "NOI_50": NOI_50,"NOI_60": NOI_60,"NOI_80": NOI_80,"NOI_99": NOI_99,"NOI_std": NOI_std, "PAR_01": PAR_01, 
                   "PAR_20": PAR_20,"PAR_40": PAR_40,"PAR_50": PAR_50,"PAR_60": PAR_60,"PAR_80": PAR_80,"PAR_99": PAR_99,
                   "PAR_std": PAR_std, "WMC_01": WMC_01, "WMC_20": WMC_20,"WMC_40": WMC_40,"WMC_50": WMC_50,"WMC_60": WMC_60,
                   "WMC_80": WMC_80,"WMC_99": WMC_99,"WMC_std": WMC_std, "NOC_01": NOC_01, "NOC_20": NOC_20,"NOC_40": NOC_40,
                   "NOC_50": NOC_50,"NOC_60": NOC_60,"NOC_80": NOC_80,"NOC_99": NOC_99,"NOC_std": NOC_std, "RMA_01": RMA_01, 
                   "RMA_20": RMA_20,"RMA_40": RMA_40,"RMA_50": RMA_50,"RMA_60": RMA_60,"RMA_80": RMA_80,"RMA_99": RMA_99,
                   "RMA_std": RMA_std, "NORM_01": NORM_01, "NORM_20": NORM_20,"NORM_40": NORM_40,"NORM_50": NORM_50,
                   "NORM_60": NORM_60,"NORM_80": NORM_80,"NORM_99": NORM_99,"NORM_std": NORM_std, "MLOC_01": MLOC_01, 
                   "MLOC_20": MLOC_20,"MLOC_40": MLOC_40,"MLOC_50": MLOC_50,"MLOC_60": MLOC_60,"MLOC_80": MLOC_80,
                   "MLOC_99": MLOC_99,"MLOC_std": MLOC_std, "NSC_01": NSC_01, "NSC_20": NSC_20,"NSC_40": NSC_40,"NSC_50": NSC_50,
                   "NSC_60": NSC_60,"NSC_80": NSC_80,"NSC_99": NSC_99,"NSC_std": NSC_std, "RMD_01": RMD_01, "RMD_20": RMD_20,
                   "RMD_40": RMD_40,"RMD_50": RMD_50,"RMD_60": RMD_60,"RMD_80": RMD_80,"RMD_99": RMD_99,"RMD_std": RMD_std, 
                   "NOF_01": NOF_01, "NOF_20": NOF_20,"NOF_40": NOF_40,"NOF_50": NOF_50,"NOF_60": NOF_60,"NOF_80": NOF_80,
                   "NOF_99": NOF_99,"NOF_std": NOF_std, "VG_01": VG_01, "VG_20": VG_20,"VG_40": VG_40,"VG_50": VG_50,
                   "VG_60": VG_60,"VG_80": VG_80,"VG_99": VG_99,"VG_std": VG_std})
score_df = pd.DataFrame.from_records(scores)
score_df

,Project,LCOM_01,LCOM_20,LCOM_40,LCOM_50,LCOM_60,LCOM_80,LCOM_99,LCOM_std,DIT_01,DIT_20,DIT_40,DIT_50,DIT_60,DIT_80,DIT_99,DIT_std,CE_01,CE_20,CE_40,CE_50,CE_60,CE_80,CE_99,CE_std,RMI_01,RMI_20,RMI_40,RMI_50,RMI_60,RMI_80,RMI_99,RMI_std,NOM_01,NOM_20,NOM_40,NOM_50,NOM_60,NOM_80,NOM_99,NOM_std,NSF_01,NSF_20,NSF_40,NSF_50,NSF_60,NSF_80,NSF_99,NSF_std,NSM_01,NSM_20,NSM_40,NSM_50,NSM_60,NSM_80,NSM_99,NSM_std,NBD_01,NBD_20,NBD_40,NBD_50,NBD_60,NBD_80,NBD_99,NBD_std,SIX_01,SIX_20,SIX_40,SIX_50,SIX_60,SIX_80,SIX_99,SIX_std,CA_01,CA_20,CA_40,CA_50,CA_60,CA_80,CA_99,CA_std,NOI_01,NOI_20,NOI_40,NOI_50,NOI_60,NOI_80,NOI_99,NOI_std,PAR_01,PAR_20,PAR_40,PAR_50,PAR_60,PAR_80,PAR_99,PAR_std,WMC_01,WMC_20,WMC_40,WMC_50,WMC_60,WMC_80,WMC_99,WMC_std,NOC_01,NOC_20,NOC_40,NOC_50,NOC_60,NOC_80,NOC_99,NOC_std,RMA_01,RMA_20,RMA_40,RMA_50,RMA_60,RMA_80,RMA_99,RMA_std,NORM_01,NORM_20,NORM_40,NORM_50,NORM_60,NORM_80,NORM_99,NORM_std,MLOC_01,MLOC_20,MLOC_40,MLOC_50,MLOC_60,MLOC_80,MLOC_99,MLOC_std,NSC_01,NSC_20,NSC_40,NSC_50,NSC_60,NSC_80,NSC_99,NSC_std,RMD_01,RMD_20,RMD_40,RMD_50,RMD_60,RMD_80,RMD_99,RMD_std,NOF_01,NOF_20,NOF_40,NOF_50,NOF_60,NOF_80,NOF_99,NOF_std,VG_01,VG_20,VG_40,VG_50,VG_60,VG_80,VG_99,VG_std
0,ant-1.8,0.0,0.0,0.0,0.000,0.0000,0.6670,0.95881,0.349370,0.00,1.0,2.0,2.0,3.0,4.0,5.00,1.402911,0.00,1.0,2.0,2.0,4.0,10.0,72.31,14.612170,0.00000,0.5710,1.0000,1.0000,1.0000,1.0000,1.00000,0.313510,0.00,1.0,3.0,4.0,5.0,11.0,46.81,0.349370,0.0,0.0,0.0,0.0,0.0,1.0,14.00,2.904167,0.0,0.0,0.0,0.0,0.0,0.0,8.81,1.712138,0.0,1.0,1.0,1.0,1.0,2.0,5.00,0.990977,0.0,0.0,0.0000,0.000,0.067,0.6000,3.00000,0.649377,0.00,0.0,0.0,0.0,0.0,4.0,340.37,69.706238,0.00,0.0,0.0,0.0,0.0,1.0,9.19,1.971234,0.0,0.0,0.0,1.0,1.0,1.0,4.00,0.926909,0.00,2.0,5.0,7.0,10.0,23.0,134.86,27.414184,1.00,1.0,3.0,4.0,6.0,14.0,105.74,28.016630,0.00000,0.0000,0.0000,0.0000,0.0456,0.1726,0.62358,0.147081,0.0,0.0,0.0,0.0,1.0,1.0,5.00,1.276106,0.0,1.0,1.0,1.0,3.0,8.0,62.00,12.364365,0.0,0.0,0.0,0.0,0.0,0.0,13.00,6.027298,0.00000,0.0000,0.0000,0.0000,0.1128,0.3330,0.89548,0.258122,0.0,0.0,0.0,1.0,1.0,4.0,22.00,4.528603,1.0,1.0,1.0,1.0,1.0,2.0,15.00,3.005299
1,antlr-3,0.0,0.0,0.0,0.000,0.0000,0.5000,0.97942,0.315929,0.00,1.0,1.0,2.0,2.0,3.0,5.71,1.215004,0.19,1.0,2.0,3.0,6.0,15.6,37.05,11.037734,0.04008,0.1822,0.3952,0.5040,0.7530,1.0000,1.00000,0.379568,0.00,1.0,2.0,4.0,5.0,14.0,57.42,0.315929,0.0,0.0,0.0,0.0,0.0,0.0,16.71,7.835914,0.0,0.0,0.0,0.0,0.0,0.0,6.71,2.642318,0.0,1.0,1.0,1.0,1.0,2.0,5.00,0.909568,0.0,0.0,0.0000,0.000,0.000,0.3330,3.00000,0.722413,0.00,0.0,2.0,2.5,8.0,30.0,79.11,22.951665,0.00,0.0,0.0,0.0,0.4,1.2,6.62,1.863782,0.0,0.0,0.0,0.0,1.0,1.0,5.00,1.087519,0.00,1.0,4.0,5.0,9.0,29.2,155.91,36.786419,0.19,1.8,9.2,14.0,19.0,38.6,62.63,19.157175,0.00000,0.0000,0.0046,0.0430,0.0732,0.1560,0.29160,0.095694,0.0,0.0,0.0,0.0,0.0,1.0,8.00,2.169242,0.0,1.0,2.0,5.0,7.0,14.0,69.00,13.293574,0.0,0.0,0.0,0.0,0.0,0.0,26.62,10.178750,0.00000,0.0138,0.1222,0.4000,0.5834,0.6758,0.94260,0.340022,0.0,0.0,0.0,1.0,1.0,3.0,29.84,5.267432,1.0,1.0,1.0,1.0,1.0,2.0,15.00,2.931288
2,aoi-2.8,0.0,0.0,0.0,0.000,0.3330,0.6670,0.92600,0.332304,0.00,1.0,1.0,1.0,2.0,2.0,7.00,1.618856,0.22,5.0,9.8,10.0,15.0,32.2,73.60,19.385623,0.00154,0.1564,0.3632,0.6250,0.7258,1.0000,1.00000,0.381112,0.00,1.0,2.0,4.0,6.0,12.0,44.00,0.332304,0.0,0.0,0.0,0.0,0.0,1.0,14.77,3.166593,0.0,0.0,0.0,0.0,0.0,0.0,14.36,2.600284,0.0,1.0,1.0,1.0,1.0,2.0,5.00,0.940688,0.0,0.0,0.0000,0.000,0.000,0.5426,4.00000,0.776029,0.00,0.0,4.0,9.0,23.0,143.8,275.00,95.388596,0.00,0.0,0.8,1.0,1.0,2.6,5.78,1.725191,0.0,0.0,1.0,1.0,1.0,2.0,8.00,1.777643,0.00,1.0,5.0,10.0,15.0,37.0,271.59,55.560924,1.22,9.0,12.0,16.0,18.6,48.0,89.16,25.134578,0.00000,0.0000,0.0584,0.0860,0.1280,0.1532,0.30066,0.086375,0.0,0.0,0.0,0.0,0.0,1.0,14.00,2.905687,0.0,1.0,2.0,3.0,5.0,15.0,121.00,28.548488,0.0,0.0,0.0,0.0,0.0,0.0,9.00,2.533198,0.00000,0.0392,0.1808,0.2320,0.5308,0.7150,0.95486,0.333972,0.0,0.0,0.0,2.0,3.0,9.0,30.00,7.041553,1.0,1.0,1.0,1.0,2.0,4.0,31.00,6.5979

In [26]:
Qcc_quant = pd.merge(Qcc_df, score_df, on="Project")

In [27]:
Qcc_quant

,Project,Domain,NOP,TLOC,LCOM_01,LCOM_20,LCOM_40,LCOM_50,LCOM_60,LCOM_80,LCOM_99,LCOM_std,DIT_01,DIT_20,DIT_40,DIT_50,DIT_60,DIT_80,DIT_99,DIT_std,CE_01,CE_20,CE_40,CE_50,CE_60,CE_80,CE_99,CE_std,RMI_01,RMI_20,RMI_40,RMI_50,RMI_60,RMI_80,RMI_99,RMI_std,NOM_01,NOM_20,NOM_40,NOM_50,NOM_60,NOM_80,NOM_99,NOM_std,NSF_01,NSF_20,NSF_40,NSF_50,NSF_60,NSF_80,NSF_99,NSF_std,NSM_01,NSM_20,NSM_40,NSM_50,NSM_60,NSM_80,NSM_99,NSM_std,NBD_01,NBD_20,NBD_40,NBD_50,NBD_60,NBD_80,NBD_99,NBD_std,SIX_01,SIX_20,SIX_40,SIX_50,SIX_60,SIX_80,SIX_99,SIX_std,CA_01,CA_20,CA_40,CA_50,CA_60,CA_80,CA_99,CA_std,NOI_01,NOI_20,NOI_40,NOI_50,NOI_60,NOI_80,NOI_99,NOI_std,PAR_01,PAR_20,PAR_40,PAR_50,PAR_60,PAR_80,PAR_99,PAR_std,WMC_01,WMC_20,WMC_40,WMC_50,WMC_60,WMC_80,WMC_99,WMC_std,NOC_01,NOC_20,NOC_40,NOC_50,NOC_60,NOC_80,NOC_99,NOC_std,RMA_01,RMA_20,RMA_40,RMA_50,RMA_60,RMA_80,RMA_99,RMA_std,NORM_01,NORM_20,NORM_40,NORM_50,NORM_60,NORM_80,NORM_99,NORM_std,MLOC_01,MLOC_20,MLOC_40,MLOC_50,MLOC_60,MLOC_80,MLOC_99,MLOC_std,NSC_01,NSC_20,NSC_40,NSC_50,NSC_60,NSC_80,NSC_99,NSC_std,RMD_01,RMD_20,RMD_40,RMD_50,RMD_60,RMD_80,RMD_99,RMD_std,NOF_01,NOF_20,NOF_40,NOF_50,NOF_60,NOF_80,NOF_99,NOF_std,VG_01,VG_20,VG_40,VG_50,VG_60,VG_80,VG_99,VG_std
0,ant-1.8,parsers/generators/make,127.0,127559.0,0.0,0.0,0.0,0.000,0.0000,0.6670,0.95881,0.349370,0.00,1.0,2.0,2.0,3.0,4.0,5.00,1.402911,0.00,1.0,2.0,2.0,4.0,10.0,72.31,14.612170,0.00000,0.5710,1.0000,1.0000,1.0000,1.0000,1.00000,0.313510,0.00,1.0,3.0,4.0,5.0,11.0,46.81,0.349370,0.0,0.0,0.0,0.0,0.0,1.0,14.00,2.904167,0.0,0.0,0.0,0.0,0.0,0.0,8.81,1.712138,0.0,1.0,1.0,1.0,1.0,2.0,5.00,0.990977,0.0,0.0,0.0000,0.000,0.067,0.6000,3.00000,0.649377,0.00,0.0,0.0,0.0,0.0,4.0,340.37,69.706238,0.00,0.0,0.0,0.0,0.0,1.0,9.19,1.971234,0.0,0.0,0.0,1.0,1.0,1.0,4.00,0.926909,0.00,2.0,5.0,7.0,10.0,23.0,134.86,27.414184,1.00,1.0,3.0,4.0,6.0,14.0,105.74,28.016630,0.00000,0.0000,0.0000,0.0000,0.0456,0.1726,0.62358,0.147081,0.0,0.0,0.0,0.0,1.0,1.0,5.00,1.276106,0.0,1.0,1.0,1.0,3.0,8.0,62.00,12.364365,0.0,0.0,0.0,0.0,0.0,0.0,13.00,6.027298,0.00000,0.0000,0.0000,0.0000,0.1128,0.3330,0.89548,0.258122,0.0,0.0,0.0,1.0,1.0,4.0,22.00,4.528603,1.0,1.0,1.0,1.0,1.0,2.0,15.00,3.005299
1,antlr-3,parsers/generators/make,20.0,47443.0,0.0,0.0,0.0,0.000,0.0000,0.5000,0.97942,0.315929,0.00,1.0,1.0,2.0,2.0,3.0,5.71,1.215004,0.19,1.0,2.0,3.0,6.0,15.6,37.05,11.037734,0.04008,0.1822,0.3952,0.5040,0.7530,1.0000,1.00000,0.379568,0.00,1.0,2.0,4.0,5.0,14.0,57.42,0.315929,0.0,0.0,0.0,0.0,0.0,0.0,16.71,7.835914,0.0,0.0,0.0,0.0,0.0,0.0,6.71,2.642318,0.0,1.0,1.0,1.0,1.0,2.0,5.00,0.909568,0.0,0.0,0.0000,0.000,0.000,0.3330,3.00000,0.722413,0.00,0.0,2.0,2.5,8.0,30.0,79.11,22.951665,0.00,0.0,0.0,0.0,0.4,1.2,6.62,1.863782,0.0,0.0,0.0,0.0,1.0,1.0,5.00,1.087519,0.00,1.0,4.0,5.0,9.0,29.2,155.91,36.786419,0.19,1.8,9.2,14.0,19.0,38.6,62.63,19.157175,0.00000,0.0000,0.0046,0.0430,0.0732,0.1560,0.29160,0.095694,0.0,0.0,0.0,0.0,0.0,1.0,8.00,2.169242,0.0,1.0,2.0,5.0,7.0,14.0,69.00,13.293574,0.0,0.0,0.0,0.0,0.0,0.0,26.62,10.178750,0.00000,0.0138,0.1222,0.4000,0.5834,0.6758,0.94260,0.340022,0.0,0.0,0.0,1.0,1.0,3.0,29.84,5.267432,1.0,1.0,1.0,1.0,1.0,2.0,15.00,2.931288
2,aoi-2.8,3D/graphics/media,23.0,110009.0,0.0,0.0,0.0,0.000,0.3330,0.6670,0.92600,0.332304,0.00,1.0,1.0,1.0,2.0,2.0,7.00,1.618856,0.22,5.0,9.8,10.0,15.0,32.2,73.60,19.385623,0.00154,0.1564,0.3632,0.6250,0.7258,1.0000,1.00000,0.381112,0.00,1.0,2.0,4.0,6.0,12.0,44.00,0.332304,0.0,0.0,0.0,0.0,0.0,1.0,14.77,3.166593,0.0,0.0,0.0,0.0,0.0,0.0,14.36,2.600284,0.0,1.0,1.0,1.0,1.0,2.0,5.00,0.940688,0.0,0.0,0.0000,0.000,0.000,0.5426,4.00000,0.776029,0.00,0.0,4.0,9.0,23.0,143.8,275.00,95.388596,0.00,0.0,0.8,1.0,1.0,2.6,5.78,1.725191,0.0,0.0,1.0,1.0,1.0,2.0,8.00,1.777643,0.00,1.0,5.0,10.0,15.0,37.0,271.59,55.560924,1.22,9.0,12.0,16.0,18.6,48.0,89.16,25.134578,0.00000,0.0000,0.0584,0.0860,0.1280,0.1532,0.30066,0.086375,0.0,0.0,0.0,0.0,0.0,1.0,14.00,2.905687,0.0,1.0,2.0,3.0,5.0,15.0,121.00,28.548488,0.0,0.0,0.0,0.0,0.0,0.0,9.00,2.533198,0.00000,0.0

In [28]:
Qcc_quant.loc[:, ['Project', 'Domain', 'MLOC_01', 'MLOC_20', 'MLOC_40', 'MLOC_50', 'MLOC_60', 'MLOC_80', 'MLOC_99', 'MLOC_std']]

,Project,Domain,MLOC_01,MLOC_20,MLOC_40,MLOC_50,MLOC_60,MLOC_80,MLOC_99,MLOC_std
0,ant-1.8,parsers/generators/make,0.0,1.0,1.0,1.0,3.0,8.0,62.00,12.364365
1,antlr-3,parsers/generators/make,0.0,1.0,2.0,5.0,7.0,14.0,69.00,13.293574
2,aoi-2.8,3D/graphics/media,0.0,1.0,2.0,3.0,5.0,15.0,121.00,28.548488
3,argouml,diagram generator/data visualization,0.0,1.0,1.0,3.0,4.0,10.0,64.00,15.348037
4,aspectj,programming language,0.0,1.0,1.0,2.0,4.0,10.0,86.00,23.072296
5,axion-1,database,0.0,1.0,1.0,1.0,2.0,7.0,44.00,11.578342
6,azureus,database,0.0,0.0,1.0,1.0,2.0,10.0,108.00,24.873454
7,batik-1,3D/graphics/media,0.0,1.0,1.0,2.0,4.0,9.0,80.38,21.764576
8,castor-,middleware,0.0,1.0,1.0,2.0,3.0,7.0,74.60,19.605149
9,cayenne,database,0.0,1.0,1.0,2.0,3.0,10.0,50.00,12.199007


In [29]:
Qcc_quant.to_csv('Qcc_quant.csv',index=False)

In [30]:
Qcc_quant.isnull().values.any()

False

## TD data set

In [57]:
TD = pd.DataFrame({'Project':sq.Project,"SonarQube_debt": sq.sqale_index,"Codiga_debt":co.Time,"CodeClimate_debt":cc['TD time (hrs)']})

In [58]:
TD

,Project,SonarQube_debt,Codiga_debt,CodeClimate_debt
0,ant-1.8,1431.033333,677.56,5595.5
1,antlr-3,934.400000,737.00,5701.4
2,aoi-2.8,1606.300000,702.66,10842.6
3,argouml,2277.266667,992.36,11998.6
4,aspectj,6653.583333,3144.50,67848.8
5,axion-1,320.850000,197.36,1562.4
6,azureus,5155.616667,2916.10,30520.2
7,batik-1,2999.200000,1436.56,12370.0
8,castor-,21213.083330,1155.66,5376.4
9,cayenne,2877.066667,830.26,10111.4


In [59]:
TD.to_csv('TD.csv',index=False)

# QCC_sq

In [60]:
x_sq = sq.drop(['lines', 'code_smells','sqale_index'], axis=1)

In [61]:
QCC_sq = pd.merge(Qcc_df, x_sq, on="Project")

In [62]:
QCC_sq = Qcc_df

In [63]:
QCC_sq

,Project,Domain,NOP,TLOC
0,ant-1.8,parsers/generators/make,127.0,127559.0
1,antlr-3,parsers/generators/make,20.0,47443.0
2,aoi-2.8,3D/graphics/media,23.0,110009.0
3,argouml,diagram generator/data visualization,77.0,105806.0
4,aspectj,programming language,144.0,501762.0
5,axion-1,database,13.0,24163.0
6,azureus,database,473.0,495532.0
7,batik-1,3D/graphics/media,117.0,194715.0
8,castor-,middleware,381.0,219681.0
9,cayenne,database,235.0,198176.0


In [64]:
QCC_sq.to_csv('QCC_sq.csv',index=False)

# QCC_co

In [65]:
x_co = co.drop(['LOC','Total Cost','Time'], axis=1)

In [66]:
x_co['Grade'].replace(['A', 'B','C','D','E','F'],[5,4,3,2,1,0], inplace=True)

In [67]:
QCC_co = pd.merge(Qcc_df, x_co, on="Project")

In [68]:
QCC_co

,Project,Domain,NOP,TLOC,Quality Score(%),Grade,Errors reported,Violations,Informational,Error,Warning,Critical,Long function(%),Complex function(%),Duplicated lines(%),Outdated,To upgrade,Up to Date
0,ant-1.8,parsers/generators/make,127.0,127559.0,94.0,5,4213,3691,189,189,3218,95,4.0,1.0,0.0,0,0,0
1,antlr-3,parsers/generators/make,20.0,47443.0,93.0,5,3582,3184,499,968,1405,312,5.0,1.0,2.0,0,0,0
2,aoi-2.8,3D/graphics/media,23.0,110009.0,85.0,5,2972,2083,275,159,1514,135,10.0,2.0,9.0,0,0,0
3,argouml,diagram generator/data visualization,77.0,105806.0,95.0,5,5270,4440,156,281,3736,267,5.0,1.0,0.0,0,0,0
4,aspectj,programming language,144.0,501762.0,88.0,5,19687,17330,1648,770,14205,707,8.0,2.0,0.0,0,0,0
5,axion-1,database,13.0,24163.0,85.0,5,1346,1256,26,29,1136,65,3.0,1.0,2.0,0,0,0
6,azureus,database,473.0,495532.0,93.0,5,14526,11430,681,752,9546,451,10.0,1.0,0.0,0,0,0
7,batik-1,3D/graphics/media,117.0,194715.0,90.0,5,8981,8052,302,437,6983,330,6.0,1.0,0.0,0,0,0
8,castor-,middleware,381.0,219681.0,95.0,5,6633,55811,312,317,4814,138,5.0,1.0,0.0,0,0,0
9,cayenne,database,235.0,198176.0,97.0,5,4021,3288,157,205,2625,301,4.0,1.0,4.0,0,0,0


In [69]:
QCC_co.to_csv('QCC_co.csv',index=False)

# QCC_cc

In [70]:
x_cc = cc.drop(['LOC','Code smell','TD time (hrs)'], axis=1)

In [71]:
x_cc['Maintainibility score'].replace(['A', 'B','C','D','E','F'],[5,4,3,2,1,0], inplace=True)

In [72]:
QCC_cc = pd.merge(Qcc_df, x_cc, on="Project")

In [73]:
QCC_cc

,Project,Domain,NOP,TLOC,Files,Duplication,Issues,Maintainibility score,TD ratio (%)
0,ant-1.8,parsers/generators/make,127.0,127559.0,3044,758,2927,3,18.7
1,antlr-3,parsers/generators/make,20.0,47443.0,1600,986,2180,2,24.1
2,aoi-2.8,3D/graphics/media,23.0,110009.0,1915,2470,5172,2,33.4
3,argouml,diagram generator/data visualization,77.0,105806.0,6241,2428,6475,2,20.2
4,aspectj,programming language,144.0,501762.0,10951,6697,22862,2,43.1
5,axion-1,database,13.0,24163.0,540,322,985,2,24.0
6,azureus,database,473.0,495532.0,11329,3911,12859,3,19.7
7,batik-1,3D/graphics/media,117.0,194715.0,5647,2720,6471,2,21.7
8,castor-,middleware,381.0,219681.0,3709,935,2290,2,23.1
9,cayenne,database,235.0,198176.0,5426,2071,4990,2,22.6


In [74]:
QCC_cc.to_csv('QCC_cc.csv',index=False)

# Qcc_sq_co_cc

In [75]:
Qcc_sq_co_cc = pd.merge(Qcc_df, x_sq, on="Project")
Qcc_sq_co_cc = pd.merge(Qcc_sq_co_cc, x_co, on="Project")
Qcc_sq_co_cc = pd.merge(Qcc_sq_co_cc, x_cc, on="Project")

In [76]:
Qcc_sq_co_cc

,Project,Domain,NOP,TLOC,bugs,reliability_rating,reliability_remediation_effort,vulnerabilities,security_rating,security_remediation_effort,security_hotspots,security_review_rating,sqale_rating,sqale_debt_ratio,files,functions,statements,comment_lines,duplicated_files,duplicated_lines,duplicated_lines_density,complexity,cognitive_complexity,violations,Quality Score(%),Grade,Errors reported,Violations,Informational,Error,Warning,Critical,Long function(%),Complex function(%),Duplicated lines(%),Outdated,To upgrade,Up to Date,Files,Duplication,Issues,Maintainibility score,TD ratio (%)
0,ant-1.8,parsers/generators/make,127.0,127559.0,404,5,84.966667,17,5,5.750000,86,5,1,1.6,1725,13160,59383,57876,156,10111,3.1,26923,22931,9513,94.0,5,4213,3691,189,189,3218,95,4.0,1.0,0.0,0,0,0,3044,758,2927,3,18.7
1,antlr-3,parsers/generators/make,20.0,47443.0,106,5,14.950000,0,1,0.000000,80,5,1,2.0,1421,7613,44089,22153,94,11499,5.7,14587,9305,6775,93.0,5,3582,3184,499,968,1405,312,5.0,1.0,2.0,0,0,0,1600,986,2180,2,24.1
2,aoi-2.8,3D/graphics/media,23.0,110009.0,1953,5,450.116667,5,5,1.416667,165,5,1,2.6,575,6839,67724,14285,226,27481,16.6,23623,28934,14083,85.0,5,2972,2083,275,159,1514,135,10.0,2.0,9.0,0,0,0,1915,2470,5172,2,33.4
3,argouml,diagram generator/data visualization,77.0,105806.0,493,5,59.650000,8,5,1.916667,60,5,1,1.0,3001,18026,89295,106516,352,25667,3.4,42765,37119,17661,95.0,5,5270,4440,156,281,3736,267,5.0,1.0,0.0,0,0,0,6241,2428,6475,2,20.2
4,aspectj,programming language,144.0,501762.0,859,5,144.133333,8,5,2.000000,193,5,1,2.7,2929,43595,271309,146652,1514,310623,39.5,129113,151894,45511,88.0,5,19687,17330,1648,770,14205,707,8.0,2.0,0.0,0,0,0,10951,6697,22862,2,43.1
5,axion-1,database,13.0,24163.0,125,5,15.816667,0,1,0.000000,8,5,1,2.6,262,3000,11593,3394,34,1595,3.7,6224,5343,3103,85.0,5,1346,1256,26,29,1136,65,3.0,1.0,2.0,0,0,0,540,322,985,2,24.0
6,azureus,database,473.0,495532.0,1317,5,226.250000,42,5,14.783333,482,5,1,2.1,3268,35670,212707,47432,490,32075,4.0,86381,104075,54836,93.0,5,14526,11430,681,752,9546,451,10.0,1.0,0.0,0,0,0,11329,3911,12859,3,19.7
7,batik-1,3D/graphics/media,117.0,194715.0,543,5,99.583333,0,1,0.000000,105,5,1,2.8,1851,16568,90615,62443,323,25802,6.5,38324,34866,14598,90.0,5,8981,8052,302,437,6983,330,6.0,1.0,0.0,0,0,0,5647,2720,6471,2,21.7
8,castor-,middleware,381.0,219681.0,88540,5,22706.366670,6,5,2.000000,234,5,2,5.6,3414,6120,39288,78414,2139,371888,39.4,15711,14227,338614,95.0,5,6633,55811,312,317,4814,138,5.0,1.0,0.0,0,0,0,3709,935,2290,2,23.1
9,cayenne,database,235.0,198176.0,1080,5,148.716667,0,1,0.000000,147,5,1,1.4,4868,23500,117840,84388,955,88488,12.7,46324,36577,23702,97.0,5,4021,3288,157,205,2625,301,4.0,1.0,4.0,0,0,0,5426,2071,4990,2,22.6


In [77]:
Qcc_sq_co_cc.to_csv('Qcc_sq_co_cc.csv',index=False)